<a href="https://colab.research.google.com/github/mmilannaik/BigOCheatSheet/blob/master/FastAPi_Pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Why Pydantic

In [17]:
pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.2 MB/s eta 0:00:00


In [47]:
from pydantic import BaseModel,EmailStr,AnyUrl,Field,field_validator,model_validator,computed_field
from typing import List,Dict,Optional,Annotated

In [27]:
class Patient(BaseModel):
  name : str
  age :int

def insert_patient_data(patient: Patient):
  print(patient.name)
  print(patient.age)
  print('inserted')

patient_info = {'name':'nitish','age':30}

patient1 = Patient(**patient_info)

insert_patient_data(patient1)

nitish
30
inserted


It helps in data validation and type validation

# New Pydantic model

In [53]:
class Patient(BaseModel):
  name : Annotated[str,Field(max_lenth = 50,title ='Name of patient',
                             description='Provide the name of patient in less than 50 chars',examples = ['Nitish','Amit'])]
  email : EmailStr
  linkedin_url : AnyUrl
  age :int =Field(gt=0,lt=140)
  weight: float = Field(gt=0)
  height : float
  married: bool = False
  allergies: Optional[List[str]] = None
  contact_details: Dict[str,str]


  @field_validator('email')
  @classmethod
  def email_validator(cls,value):
    valid_domains = ['hdfc.com','icici.com']
    #abc@gmail.com
    domain_name = value.split('@')[-1]
    if domain_name not in valid_domains:
      raise ValueError('Not a valid domain')
    return value

  @field_validator('name')
  @classmethod
  def transform_name(cls,value):
    return value.upper()

  @field_validator('age',mode='after')
  @classmethod
  def validate_age(cls,value):
    if 0<value<140:
      return value
    else:
      raise ValueError('Age shpuld be in between 0 and 100')

  @model_validator(mode='after')
  def validate_emergency_contact(cls,model):
    if model.age>60 and 'emergency' not in model.contact_details:
      raise ValueError('Patients older than 60 must have an emergency contact')
    return model

  @computed_field
  @property
  def bmi(self)-> float:
    bmi = round(self.weight/(self.height**2),2)
    return bmi

def insert_patient_data(patient: Patient):
  print(patient.name)
  print(patient.age)
  print(patient.allergies)
  print('BMI', patient.bmi)
  print('inserted')

patient_info = {'name':'nitish','age':130,'weight':75,'married':True,'email':'abc@hdfc.com',
                'linkedin_url':"https://linkedin.com/abc123",'height':1.75,
                'contact_details':{'email':'abc@gmail.com','phone':'2353462','emergency':'12345'}}

patient1 = Patient(**patient_info)

insert_patient_data(patient1)


NITISH
130
None
BMI 24.49
inserted


# Nested Models

In [55]:
from pydantic import BaseModel

class Address(BaseModel):

    city: str
    state: str
    pin: str

class Patient(BaseModel):

    name: str
    gender: str
    age: int
    address: Address

address_dict = {'city': 'gurgaon', 'state': 'haryana', 'pin': '122001'}

address1 = Address(**address_dict)

patient_dict = {'name': 'nitish', 'gender': 'male', 'age': 35, 'address': address1}

patient1 = Patient(**patient_dict)

temp = patient1.model_dump()

print(type(temp))

<class 'dict'>
